In [1]:
import torch
import torch as t
import torch.utils.data as Data
from torch import nn
from torch.autograd import Variable
import numpy as np
import pandas as pd
import jieba
import gensim
from gensim.models import Word2Vec, FastText
import re
import os
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import accuracy_score
import copy

from m import f1_for_car, BOW, BasicModule

In [2]:
embedding_dim = 300
USE_CUDA=True
EPOCH = 20           # 训练整批数据多少次
BATCH_SIZE = 128
LR = 0.002         # 学习率

# 1. 构造embedding字典

In [3]:
# 以训练数据为例
data_path_dir = 'data'
data = pd.read_csv(os.path.join(data_path_dir,'cuishou_intent3.csv'),sep='\t')
data.columns = ['content','label']

data_tmp = data.copy(deep=True)

# label数值化(编码)
d_ = {}
for key, value in enumerate(set(data_tmp.label)):
    d_[value] = key
data_tmp['label'] = data_tmp['label'].apply(lambda x : d_.get(x))

y_all = np.array(data_tmp.label.tolist())
# 构造embedding字典
bow = BOW(data_tmp.content.apply(jieba.lcut).tolist(), min_count=1, maxlen=30) # 长度补齐或截断固定长度30

vocab_size = len(bow.word2idx)
word2vec = gensim.models.KeyedVectors.load_word2vec_format('data/ft_wv.txt')

embedding_matrix = np.zeros((vocab_size+1,300))
for key, value in bow.word2idx.items():
    if key in word2vec.vocab: # Word2Vec训练得到的的实例需要word2vec.wv.vocab
        embedding_matrix[value] = word2vec.get_vector(key)
    else:
        embedding_matrix[value] = [0] * embedding_dim

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.963 seconds.
Prefix dict has been built succesfully.
Doc To Number: 100%|██████████| 53850/53850 [00:00<00:00, 150507.28it/s]


In [4]:
# np.save('save/embedding_matrix',arr=embedding_matrix)

# 2. 数据处理 train_test_split 装入tensor

In [4]:
X = copy.deepcopy(bow.doc2num)
y = copy.deepcopy(y_all)
skf = StratifiedKFold(n_splits=5,shuffle=True)
for train_idx, val_idx in skf.split(X,y):
    pass

X_train = X[train_idx]
y_train = y[train_idx]
X_val = X[val_idx]
y_val = y[val_idx]

# 数据处理成tensor
train_label_tensor = torch.from_numpy(np.array(y_train)).long()
train_content_tensor = torch.from_numpy(np.array(X_train)).long()

train_torch_dataset = Data.TensorDataset(train_content_tensor, train_label_tensor)
train_loader = Data.DataLoader(
        dataset=train_torch_dataset,      # torch TensorDataset format
        batch_size=BATCH_SIZE,      # mini batch size
        shuffle=True,               # random shuffle for training
        num_workers=8,              # subprocesses for loading data
    )

val_label_tensor = torch.from_numpy(np.array(y_val)).long()
val_content_tensor = torch.from_numpy(np.array(X_val)).long()

val_torch_dataset = Data.TensorDataset(val_content_tensor, val_label_tensor)
val_loader = Data.DataLoader(
        dataset=val_torch_dataset,      # torch TensorDataset format
        batch_size=BATCH_SIZE,      # mini batch size
        shuffle=True,               # random shuffle for training
        num_workers=8,              # subprocesses for loading data
    )

if USE_CUDA:
    val_content_tensor = val_content_tensor.cuda()
    val_label_tensor = val_label_tensor.cuda()

# 3. 构建AttLSTM模型

In [5]:
# 配置文件
class Config(object):
    '''
    并不是所有的配置都生效,实际运行中只根据需求获取自己需要的参数
    '''

    loss = 'multilabelloss'
    model='LSTMText' 
    num_classes = 21 # 类别
    embedding_dim = 300 # embedding大小
    linear_hidden_size = 100 # 全连接层隐藏元数目
    kmax_pooling = 30 # k
    hidden_size = 128 # LSTM hidden size
    num_layers= 1 # LSTM layers
    inception_dim = 256 # inception的卷积核数
    
    # vocab_size = 11973 # num of chars
    vocab_size = vocab_size # num of words 
    content_seq_len = 30 # 描述长度 word为100 char为200
    static = False
    use_pretrained_embedding = True
    bidirectional = True
    USE_CUDA = True
    embedding_path = 'save/embedding_matrix.npy'

opt = Config()

In [6]:
# 相当于把seq_len压缩成k个'词'
# dim共三个维度，这里取2即seq_len那个维度，100->k
def kmax_pooling(x, dim, k):
    index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
    return x.gather(dim, index)

class Embed_Layer(BasicModule):
    def __init__(self, embedding_matrix=None, opt=None):
        super(Embed_Layer, self).__init__()
        self.encoder = nn.Embedding(opt.vocab_size+1, opt.embedding_dim)
        if opt.use_pretrained_embedding:
#             self.encoder.weight.data.copy_(t.from_numpy(np.load(embedding_path))) # 方法一，加载np.save的npy文件
#             self.encoder.weight.data.copy_(t.from_numpy(embedding_matrix)) # 方法二
            self.encoder.weight = nn.Parameter(t.from_numpy(embedding_matrix).float(), requires_grad=False)
    def forward(self, x):
        return self.encoder(x)

class AttLSTM(BasicModule): 
    def __init__(self, opt):
        super(AttLSTM, self).__init__()
        self.model_name = 'AttLSTM'
        self.opt=opt

        self.content_lstm =nn.LSTM(input_size = opt.embedding_dim,\
                            hidden_size = opt.hidden_size,
                            num_layers = opt.num_layers,
                            bias = True,
                            batch_first = False,
                            dropout = 0.5, # dropout
                            bidirectional = opt.bidirectional
                            )

    def init_weights(self):
        ih = (param.data for name, param in self.named_parameters() if 'weight_ih' in name)
        hh = (param.data for name, param in self.named_parameters() if 'weight_hh' in name)
        b = (param.data for name, param in self.named_parameters() if 'bias' in name)
        for mat in ih:
            nn.init.xavier_uniform_(mat)
        for mat in hh:
            nn.init.orthogonal_(mat)
        for mat in b:
            nn.init.constant_(mat, 0)

    
    def forward(self, content):
        # torch.Size([128, 30, 150])
        if self.opt.static:
            content=content.detach()
        
        '''
        lstm输入的时候需要转成(seq_len, batch, embedding_dim）这种维度（用permute转）<br>
        output，每个时刻的LSTM网络的最后一层的输出，维度（seq_len, batch, hidden_size * num_directions）|双向lstm所以输出的hidden_size维度要乘以2<br>
        lstm的输出为output, (hn, cn) 的元组<br>
        这里取第一个就是output(100,64,256)，第二个是元组其中的第一个hn就是最后时刻的隐层状态hn(4,64,128)
        这里的4就是(2层num_layers*双向)lstm得到
        '''
        content_out_tmp = self.content_lstm(content.permute(1,0,2))
        
        content_out1, hidden_out1 = content_out_tmp[0].permute(1,2,0), content_out_tmp[1] #content_out_tmp[1] 如果是LSTM则是(hn,cn)
        content_conv_out = kmax_pooling((content_out1), 2, self.opt.kmax_pooling)
        return content_conv_out, hidden_out1

class Attention_Layer(BasicModule):
    def __init__(self):
        super(Attention_Layer, self).__init__()
        
    def forward(self, rnn_out, state):
        # 如果lstm有两层layer而且是双向的，则取最后一层的两个双向即state[-2:]，不然hidden会比rnn_out多两倍
        # 一般 num_layers=1 就行了
        # https://blog.csdn.net/david0611/article/details/81090294
        merged_state = torch.cat([s for s in state[-2:]], 1)
        merged_state = merged_state.unsqueeze(2)
        # (batch, seq, hidden) * (batch, hidden, 1) = (batch, seq, 1)  
        
        weights = torch.bmm(rnn_out.permute(0, 2, 1), merged_state)
#         weights = torch.einsum('bsh,bhj->bsj',(rnn_out.permute(0, 2, 1), merged_state)) # 和上式相同
        weights = torch.nn.functional.softmax(weights.squeeze(2), dim=1).unsqueeze(2)
        # (batch, hidden, seq) * (batch, seq, 1) = (batch, hidden, 1)
        return torch.bmm(rnn_out, weights).squeeze(2)

class Dense_Layer(BasicModule):
    def __init__(self, opt=opt):
        super(Dense_Layer, self).__init__()
        self.opt = opt
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(self.opt.hidden_size*2, self.opt.linear_hidden_size),
            torch.nn.Dropout(0.2), # dropout
            torch.nn.BatchNorm1d(self.opt.linear_hidden_size),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(self.opt.linear_hidden_size, self.opt.num_classes),
            torch.nn.Softmax(dim=-1)
        )
    def forward(self, x):
        reshaped = x.view(x.size(0), -1)
        softmax = self.fc((reshaped))
        return softmax
    
class Net_Main(BasicModule):
    def __init__(self, opt=opt):
        super(Net_Main, self).__init__()
        self.embed_layer = Embed_Layer(embedding_matrix, opt)
        self.attlstm = AttLSTM(opt)
        self.attlstm.init_weights() # 初始化LSTM权重参数
        self.attention = Attention_Layer() # 加入attention机制
        self.dense_layer = Dense_Layer(opt)
    def forward(self, x):
        content1 = self.embed_layer(x)
        content2, (hn_out, cn_out) = self.attlstm(content1)
        # 加入attention层
        att_out = self.attention(content2, hn_out)
        res = self.dense_layer(att_out)
        return res

# 4. 跑模型

In [8]:
# model, optimizer, loss_func
m = Net_Main(opt)
optimizer = torch.optim.Adam(m.parameters(), lr=LR)   # optimize all lstm parameters;Adam比较好用
loss_func = torch.nn.CrossEntropyLoss()   # the target label is not one-hotted 适用于多分类
if USE_CUDA:
    m.cuda()
    loss_func.cuda()

/usr/local/anaconda3/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [9]:
it = 1
for epoch in tqdm_notebook(range(EPOCH)):
    for batch_id, (data, target) in enumerate(train_loader):
        if USE_CUDA:
            data, target = data.cuda(), target.cuda() # 数据载入GPU
        output = m(data)
        loss = loss_func(output, target)
#         if it % 200 == 0:
#             print('training loss: ', loss.cpu().data.numpy().tolist())
#             print('training acc: ', accuracy_score(target.cpu().data.numpy(), np.argmax(output.cpu().data.numpy(),axis=1)))
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients
        it += 1
    val_output = m(val_content_tensor)
    print('val acc: ', accuracy_score(val_label_tensor.cpu().data.numpy(), np.argmax(val_output.cpu().data.numpy(),axis=1)))
    print('epoch {}....................................'.format(epoch))
    del val_output

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

RuntimeError: CUDA error: out of memory